In [1]:
import numpy as np
import tqdm
from tqdm import tqdm
import torch
import os
import segmentation_models_pytorch as smp

In [2]:
print(torch.cuda.is_available())
device = "cuda:0"

True


In [3]:
class DataGenerator:
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.train_data_size = (0,57)
        self.test_data_size = (57, 63)
        self.ROOT_DATA_PATH = 'Task06_Lung/Preprocessed/train/data/'
        self.ROOT_LABEL_PATH = 'Task06_Lung/Preprocessed/train/label/'
        self.TEST_DATA_PATH = 'Task06_Lung/Preprocessed/test/data/'
        self.TEST_LABEL_PATH = 'Task06_Lung/Preprocessed/test/label/'

    def __len__(self):
        for _, __, files in os.walk(self.ROOT_DATA_PATH):
            return len(files)

    def train_generate(self, size):
        while True:
            batch_count = 0
            yield_list = []
            for i in range(size):
                    ct = np.load(self.ROOT_DATA_PATH + str(i) + ".npy")
                    mask = np.load(self.ROOT_LABEL_PATH + str(i) + ".npy")

                    ct = torch.Tensor(ct)
                    mask = torch.Tensor(mask)
                    ct = ct.to(device)
                    mask = mask.to(device)
                    
                    yield_list.append([ct, mask])
                    batch_count += 1

                    if batch_count == self.batch_size:
                        yield yield_list
                        batch_count = 0
                        yield_list = []

    def test_generate(self):
        for _, __, files in os.walk(self.TEST_DATA_PATH):
            for file in files:
                ct = np.load(self.TEST_DATA_PATH + file)
                mask = np.load(self.TEST_LABEL_PATH + file)

                ct = torch.Tensor(ct)
                mask = torch.Tensor(mask)
                ct = ct.to(device)
                mask = mask.to(device)
                    
                yield [[ct, mask]]         

In [4]:
datagen = DataGenerator(batch_size=1)
iter = datagen.train_generate(datagen.__len__()) 
data = next(iter)
print(data[0][0], data[0][1])

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0040, 0.0070, 0.0113,  ..., 0.0060, 0.0110, 0.0086],
          [0.0078, 0.0035, 0.0050,  ..., 0.0116, 0.0056, 0.0084],
          [0.0054, 0.0088, 0.0122,  ..., 0.0036, 0.0081, 0.0057]]]],
       device='cuda:0') tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')


In [5]:
datagen = DataGenerator(batch_size=1)
iter = datagen.test_generate() 
data = next(iter)
print(data[0][0], data[0][1])

tensor([[[[0.0370, 0.0397, 0.0413,  ..., 0.0106, 0.0086, 0.0121],
          [0.0423, 0.0366, 0.0430,  ..., 0.0040, 0.0058, 0.0085],
          [0.0271, 0.0439, 0.0377,  ..., 0.0172, 0.0122, 0.0163],
          ...,
          [0.0567, 0.0294, 0.0593,  ..., 0.0084, 0.0119, 0.0074],
          [0.0502, 0.0574, 0.0608,  ..., 0.0118, 0.0103, 0.0052],
          [0.0594, 0.0557, 0.0480,  ..., 0.0063, 0.0095, 0.0129]]]],
       device='cuda:0') tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')


In [6]:
data[0][1]

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')

In [7]:
# for row in data[0][0]:
#     print(row)

data[0][0].shape

torch.Size([1, 1, 256, 256])

In [8]:
type(data[0][0])

torch.Tensor

In [9]:
len(data)

1

In [10]:
import torch

class DoubleConv(torch.nn.Module):
    """
    Helper Class which implements the intermediate Convolutions
    """
    def __init__(self, in_channels, out_channels):
        
        super().__init__()
        self.step = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(out_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU())
        
    def forward(self, X):
        return self.step(X)


class UNet(torch.nn.Module):
    """
    This class implements a UNet for the Segmentation
    We use 3 down- and 3 UpConvolutions and two Convolutions in each step
    """

    def __init__(self):
        """Sets up the U-Net Structure
        """
        super().__init__()
        
        
        ############# DOWN #####################
        self.layer1 = DoubleConv(1, 64)
        self.layer2 = DoubleConv(64, 128)
        self.layer3 = DoubleConv(128, 256)
        self.layer4 = DoubleConv(256, 512)

        #########################################

        ############## UP #######################
        self.layer5 = DoubleConv(512 + 256, 256)
        self.layer6 = DoubleConv(256+128, 128)
        self.layer7 = DoubleConv(128+64, 64)
        self.layer8 = torch.nn.Conv2d(64, 1, 1)
        #########################################

        self.maxpool = torch.nn.MaxPool2d(2)

    def forward(self, x):
        
        ####### DownConv 1#########
        x1 = self.layer1(x)
        x1m = self.maxpool(x1)
        ###########################
        
        ####### DownConv 2#########        
        x2 = self.layer2(x1m)
        x2m = self.maxpool(x2)
        ###########################

        ####### DownConv 3#########        
        x3 = self.layer3(x2m)
        x3m = self.maxpool(x3)
        ###########################
        
        ##### Intermediate Layer ## 
        x4 = self.layer4(x3m)
        ###########################

        ####### UpCONV 1#########        
        x5 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x4)  # Upsample with a factor of 2
        #x5 = torch.nn.ConvTranspose2d(512, 512, 2, 2)(x4)
        x5 = torch.cat([x5, x3], dim=1)  # Skip-Connection
        x5 = self.layer5(x5)
        ###########################

        ####### UpCONV 2#########        
        x6 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x5)        
        #x6 = torch.nn.ConvTranspose2d(256, 256, 2, 2)(x5)
        x6 = torch.cat([x6, x2], dim=1)  # Skip-Connection    
        x6 = self.layer6(x6)
        ###########################
        
        ####### UpCONV 3#########        
        x7 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x6)
        #x7 = torch.nn.ConvTranspose2d(128, 128, 2, 2)(x6)
        x7 = torch.cat([x7, x1], dim=1)       
        x7 = self.layer7(x7)
        x7 = np.where(x7 > 0.5, 1, 0)
        ###########################
        
        ####### Predicted segmentation#########        
        ret = self.layer8(x7)
        return ret

In [11]:
# model = UNet()
model =  smp.Unet(
    encoder_name="resnet108",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.BCEWithLogitsLoss()

In [12]:
torch.device(device=device)

device(type='cuda', index=0)

In [13]:
model = model.to(device)

In [14]:
size = datagen.__len__()

In [15]:
for epoch in range(15):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in tqdm(enumerate(datagen.train_generate(size), 0)):
        
        inputs, labels = data[0][0], data[0][1]
        # inputs = inputs.to(device)
        # labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        # outputs = outputs.cpu().numpy()
        # outputs = np.where(outputs > 0.5, 1.0, 0.0)
        # labels = labels.cpu().numpy()
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

        if i == datagen.__len__()-1:
            break


print('Finished Training')

0it [00:00, ?it/s]

2004it [02:15, 23.32it/s]

[1,  2000] loss: 0.022


3481it [03:15, 17.78it/s]
2004it [01:01, 32.38it/s]

[2,  2000] loss: 0.005


...

[14,  2000] loss: 0.002


3481it [01:47, 32.27it/s]
2004it [01:02, 32.86it/s]

[15,  2000] loss: 0.002


3481it [01:48, 32.21it/s]

Finished Training


In [16]:
class DiceScore(torch.nn.Module):
    """
    class to compute the Dice Loss
    """
    def __init__(self):
        super().__init__()

    def forward(self, pred, mask):

        #flatten label and prediction tensors
        pred = torch.flatten(pred)
        mask = torch.flatten(mask)

        counter = (pred * mask).sum()  # Counter
        print(counter)
        denum = pred.sum() + mask.sum()
        print(denum)  # denominator
        dice = (2*counter)/denum

        return dice


In [17]:
preds = []
labels = []

for i, data in tqdm(enumerate(datagen.test_generate(), 0)):
        
    input, label = data[0][0], data[0][1]
        
    with torch.no_grad():
        pred = model(input)
        
    pred = pred.cpu().numpy()
    pred = np.where(pred < 0.5, 0, 1)

    preds.append(pred)
    labels.append(label.cpu().numpy())
    
preds = np.array(preds)
labels = np.array(labels)

860it [00:17, 49.08it/s]


In [18]:
dice_score = DiceScore()(torch.from_numpy(preds), torch.from_numpy(labels))
print(f"The Val Dice Score is: {dice_score}")

tensor(10225.)
tensor(57852.7500)
The Val Dice Score is: 0.3534836173057556


In [19]:
preds

array([[[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]],



       [[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]],



       [[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]],



       ...,



       [[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]]],



       [[[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0

In [20]:
labels

array([[[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]]],



       [[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]]],



       [[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]]],



       ...,



       [[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
     

In [21]:
preds[0].sum()

0

In [22]:
labels[0].sum()

0.0